## Finite difference validation of energy density derivatives

In [ ]:
import sys; sys.path.append('..')
import energy, tensors
import numpy as np
from numpy.linalg import norm
from matplotlib import pyplot as plt

In [ ]:
def         getNeoHookean(dim): return energy.NeoHookeanYoungPoisson(dim, 200, 0.35)
def      getLinearElastic(dim): return energy.IsotropicLinearElastic(dim, 200, 0.35)
def getCRIsoLinearElastic(dim): return energy.CorotatedIsotropicLinearElastic(dim, 200, 0.35)
def        getNeoMembrane(dim): return energy.NeoHookeanMembraneYoungPoisson(200, 0.35) # Dimension is ignored...

In [ ]:
def getOrthoTensor(dim):
    if (dim == 3):
        et = tensors.ElasticityTensor3D()
        et.setOrthotropic(1, 1.5, 2, 0.3, 0.05, -0.03, 0.51, 0.73, 0.82)
    if (dim == 2):
        et = tensors.ElasticityTensor2D()
        et.setOrthotropic(1, 1.5, -0.31, 0.58)
        #et.setOrthotropic(1, 1, 0.3, 0.5)
    return et
def getCROrthoLinearElastic(dim):
    return energy.CorotatedLinearElastic(getOrthoTensor(dim))
def getStVKOrtho(dim):
    return energy.StVenantKirchhoff(getOrthoTensor(dim))
def getStVKMembrane(dim):
    # Membranes are always the same dimension, but we use the
    # `dim` parameter to enable/disable tension field theory relaxation
    if (dim == 2): return energy.StVenantKirchhoffMembrane(getOrthoTensor(2))
    return energy.RelaxedStVenantKirchhoffMembrane(getOrthoTensor(2))
def getINeoMembrane(dim):
    return energy.RelaxedIncompressibleNeoHookeanMembrane(1)
def getCRLETFTMembrane(dim):
    psi = energy.IsoCRLETensionFieldMembrane2D(E=6, nu=0.5)
    psi.relaxationEnabled = psi.smoothingEnabled = False if (dim == 2) else True
    psi.smoothingEps = 1 / 256
    return psi

In [ ]:
def getPerturb(F):
    return np.random.uniform(-1, 1, F.shape)

def fd_approx(e, f, F, dF, eps = 1e-6):
    e.setDeformationGradient(F + eps * dF)
    plus = f()
    e.setDeformationGradient(F - eps * dF)
    minus = f()
    e.setDeformationGradient(F)
    return (plus - minus) / (2 * eps)

def relerror(fd, an):
    den = norm(an)
    if (den == 0.0): den = 1
    return norm(fd - an) / den

def denergy_validation_raw(e, F, dF, eps = 1e-6):
    fd = fd_approx(e, lambda: e.energy(), F, dF, eps)
    an = e.denergy(dF)
    return fd, an
def denergy_validation(e, F, dF, eps = 1e-6):
    return relerror(*denergy_validation_raw(e, F, dF, eps))

def delta_denergy_validation_raw(e, F, dF, eps = 1e-6):
    fd = fd_approx(e, lambda: e.denergy(), F, dF, eps)
    an = e.delta_denergy(dF)
    return fd, an
def delta_denergy_validation(e, F, dF, eps = 1e-6):
    return relerror(*delta_denergy_validation_raw(e, F, dF, eps))

def delta2_denergy_validation_raw(e, F, dF_a, dF_b, eps = 1e-6):
    fd = fd_approx(e, lambda: e.delta_denergy(dF_a), F, dF_b, eps)
    an = e.delta2_denergy(dF_b, dF_a)
    return fd, an
def delta2_denergy_validation(e, F, dF_a, dF_b, eps = 1e-6):
    return relerror(*delta2_denergy_validation_raw(e, F, dF_a, dF_b, eps))

def genPlots(getEnergy):
    def test(dim):
        e = getEnergy(dim)
        F = e.getDeformationGradient()
        minDim = min(F.shape)

        dF_a = getPerturb(F)
        dF_b = getPerturb(F)
        F = getPerturb(F)
        U, S, V = np.linalg.svd(F)
        # On the transition from full tension to partial tension
        S[0] = 1.0 + 0.002
        S[1] = 1.0 - 0.001
        # Two nearly equal singular values in the small tension case
        S[0] = 1.0 + 1/512
        S[1] = 1.0 + 1/512 - 1e-5
        F = U @ np.pad(np.diag(S), [(0, F.shape[0] - F.shape[1]), (0, 0)]) @ V.T
        
        e.setDeformationGradient(F)

        epsilons = np.logspace(-3, -8, 100)
        plt.loglog(epsilons, [       denergy_validation(e, F, dF_a,       eps) for eps in epsilons], label=       'denergy')
        plt.loglog(epsilons, [ delta_denergy_validation(e, F, dF_a,       eps) for eps in epsilons], label='delta  denergy')
        try: # Some energies do not implement third derivatives
            plt.loglog(epsilons, [delta2_denergy_validation(e, F, dF_a, dF_b, eps) for eps in epsilons], label='delta2 denergy')
        except: pass
        plt.legend()
        name = e.__class__.__name__
        try: name += " (Isotropic)" if e.isIsotropic() else "  (Orthotropic)"
        except: pass
        plt.title(name)
        plt.grid()
    fig = plt.figure(figsize=(10, 4))
    for dim in [2, 3]:
        plt.subplot(1, 2, dim - 1)
        test(dim)
    plt.tight_layout()

In [ ]:
egetters = [getNeoHookean, getLinearElastic, getCRIsoLinearElastic, getCROrthoLinearElastic, getStVKOrtho, getStVKMembrane, getINeoMembrane, getNeoMembrane, getCRLETFTMembrane]
for ge in egetters:
    genPlots(ge)

### For more detailed debugging in case things go wrong:

In [ ]:
getEnergy = getCROrthoLinearElastic

In [ ]:
dim = 2
e = getEnergy(dim)
F = np.identity(dim)
e.setDeformationGradient(F)

dF_a = getPerturb(F)
dF_b = getPerturb(F)

F += 1e-3 * getPerturb(F)

# Verify symmetry
print(delta2_denergy_validation_raw(e, F, dF_a, dF_b, eps = 1e-6)[1])
print(delta2_denergy_validation_raw(e, F, dF_b, dF_a, eps = 1e-6)[1])

In [ ]:
dim = 3
quantity='sigma'
e = getEnergy(dim)
quantityGetter = getattr(e, quantity)
deltaGetter = getattr(e, f'delta_{quantity}')
eps = 1e-5
F = np.identity(dim)
F += eps * getPerturb(F)
e.setDeformationGradient(F)
dF = getPerturb(F)
an = deltaGetter(dF)
e.setDeformationGradient(F + eps * dF)
Rplus  = quantityGetter()
e.setDeformationGradient(F - eps * dF)
Rminus = quantityGetter()
fd = (Rplus - Rminus) / (2 * eps)
np.linalg.norm(fd - an)

In [ ]:
getEnergy = getStVKOrtho

In [ ]:
dim = 2
e = getEnergy(dim)
F = np.identity(dim)
e.setDeformationGradient(F)

dF_a = getPerturb(F)
dF_b = getPerturb(F)

F += 1e-3 * getPerturb(F)

In [ ]:
fd_approx(e, lambda: e.energy(), F, dF_a) * 2

In [ ]:
e.denergy(dF_a)